# Real Estate Dashboard Data Preperation

* Fix date column.
* Create new feature that contains the distance to the Sydney CBD from the unit. This variable potentially has a high correlation with the price.
* Create 2 data frames from the original data set to use for the dahsboard:
  1. Dataframe that groups the data by suburb.
  3. Dataframe that contains the correlation coefficiants between the unit price and the unit features.


In [148]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from datetime import datetime
import math
from math import sin, cos, sqrt, atan2, radians

In [149]:
df = pd.read_csv('SydneyPropSales.csv')
df

,date_sold,price,suburb,city_name,state,lat,lon,bedrooms,property_type,loc_pid,lga_pid,Total sale
0,05/07/2020 00:00,990000.0,Abbotsford,Sydney,NSW,-33.850648,151.131962,2,unit,NSW3,NSW282,NaN
1,05/22/2020 00:00,860000.0,Abbotsford,Sydney,NSW,-33.851808,151.129718,2,unit,NSW3,NSW282,NaN
2,12/12/2018 00:00,1030000.0,Abbotsford,Sydney,NSW,-33.848938,151.131197,2,unit,NSW3,NSW282,NaN
3,02/25/2019 00:00,850000.0,Abbotsford,Sydney,NSW,-33.847565,151.129123,2,unit,NSW3,NSW282,NaN
4,04/12/2019 00:00,1220000.0,Abbotsford,Sydney,NSW,-33.850851,151.129917,3,unit,NSW3,NSW282,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
27007,08/06/2019 00:00,700000.0,Zetland,Sydney,NSW,-33.905819,151.212640,1,unit,NSW4555,NSW200,NaN
27008,08/16/2019 00:00,1100000.0,Zetland,Sydney,NSW,-33.905587,151.208785,2,unit,NSW4555,NSW200,NaN
27009,08/21/2019 00:00,960000.0,Zetland,Sydney,NSW,-33.905220,151.210579,2,unit,NSW4555,NSW200,NaN
27010,08/30/2019 00:00,530000.0,Zetland,Sydney,NSW,-33.904287,151.213243,1,unit,NSW4555,NSW200,NaN


In [150]:
# convert date_time column to datetime format
df['date_sold'] = pd.to_datetime(df['date_sold'], format='%m/%d/%Y %H:%M')

# create separate date and time columns
df['date'] = df['date_sold'].apply(lambda x: x.strftime('%m/%d/%Y'))
df['time'] = df['date_sold'].apply(lambda x: x.strftime('%H:%M'))

# convert date column to correct date format
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

# create new columns for the month and year
df['month_year'] = df['date'].dt.strftime('%Y-%m')
df['year'] = df['date'].dt.year

# drop original date_time column
df = df.drop('date_sold', axis=1)
df = df.drop('time', axis=1)

In [151]:
# Define the CBD coordinates in degrees
cbd_lat = -33.8568
cbd_lon = 151.2153

# Define the function to calculate distance
def calculate_distance(lat, lon):
    R = 6371  # Radius of the Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat, lon, cbd_lat, cbd_lon])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = (math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c
    return distance

df['distance_from_cbd_km'] = df.apply(lambda row: calculate_distance(row['lat'], row['lon']), axis=1)


In [152]:
df

,price,suburb,city_name,state,lat,lon,bedrooms,property_type,loc_pid,lga_pid,Total sale,date,month_year,year,distance_from_cbd_km
0,990000.0,Abbotsford,Sydney,NSW,-33.850648,151.131962,2,unit,NSW3,NSW282,NaN,2020-05-07,2020-05,2020,7.726059
1,860000.0,Abbotsford,Sydney,NSW,-33.851808,151.129718,2,unit,NSW3,NSW282,NaN,2020-05-22,2020-05,2020,7.922300
2,1030000.0,Abbotsford,Sydney,NSW,-33.848938,151.131197,2,unit,NSW3,NSW282,NaN,2018-12-12,2018-12,2018,7.815466
3,850000.0,Abbotsford,Sydney,NSW,-33.847565,151.129123,2,unit,NSW3,NSW282,NaN,2019-02-25,2019-02,2019,8.023985
4,1220000.0,Abbotsford,Sydney,NSW,-33.850851,151.129917,3,unit,NSW3,NSW282,NaN,2019-04-12,2019-04,2019,7.912232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27007,700000.0,Zetland,Sydney,NSW,-33.905819,151.212640,1,unit,NSW4555,NSW200,NaN,2019-08-06,2019-08,2019,5.456177
27008,1100000.0,Zetland,Sydney,NSW,-33.905587,151.208785,2,unit,NSW4555,NSW200,NaN,2019-08-16,2019-08,2019,5.458103
27009,960000.0,Zetland,Sydney,NSW,-33.905220,151.210579,2,unit,NSW4555,NSW200,NaN,2019-08-21,2019-08,2019,5.401718
27010,530000.0,Zetland,Sydney,NSW,-33.904287,151.213243,1,unit,NSW4555,NSW200,NaN,2019-08-30,2019-08,2019,5.283727


In [153]:
suburb_stats = df.groupby(['suburb']).agg({
    'bedrooms': 'mean',
    'price': ['mean', 'min', 'max', 'count', 'sum'],
}).reset_index()

# Rename columns
suburb_stats.columns = ['suburb', 'sub_avg_bedrooms', 'sub_avg_price', 'sub_min_price', 'sub_max_price', 'sub_house_count', 'sub_total_sales']
# Sort by average price of each suburb
suburb_stats.sort_values(by='sub_avg_price', ascending=False)

,suburb,sub_avg_bedrooms,sub_avg_price,sub_min_price,sub_max_price,sub_house_count,sub_total_sales
311,The Rocks,2.666667,5.300000e+06,5300000.0,5300000.0,1,5300000.0
15,Barangaroo,2.000000,3.650000e+06,1975000.0,5700000.0,3,10950000.0
250,Point Piper,2.370370,3.292533e+06,980000.0,8700000.0,15,49388000.0
321,Walsh Bay,2.562500,2.701000e+06,1950000.0,3550000.0,10,27010000.0
255,Putney,3.000000,2.050000e+06,2050000.0,2050000.0,1,2050000.0
...,...,...,...,...,...,...,...
181,Lurnea,3.000000,NaN,NaN,NaN,0,0.0
226,North Willoughby,2.000000,NaN,NaN,NaN,0,0.0
234,Oyster Bay,3.000000,NaN,NaN,NaN,0,0.0
249,Plumpton,2.000000,NaN,NaN,NaN,0,0.0


In [154]:
df = pd.merge(df, suburb_stats, on='suburb')
df

,price,suburb,city_name,state,lat,lon,bedrooms,property_type,loc_pid,lga_pid,...,date,month_year,year,distance_from_cbd_km,sub_avg_bedrooms,sub_avg_price,sub_min_price,sub_max_price,sub_house_count,sub_total_sales
0,990000.0,Abbotsford,Sydney,NSW,-33.850648,151.131962,2,unit,NSW3,NSW282,...,2020-05-07,2020-05,2020,7.726059,2.428571,1.327737e+06,760000.0,3600000.0,38,50454000.0
1,860000.0,Abbotsford,Sydney,NSW,-33.851808,151.129718,2,unit,NSW3,NSW282,...,2020-05-22,2020-05,2020,7.922300,2.428571,1.327737e+06,760000.0,3600000.0,38,50454000.0
2,1030000.0,Abbotsford,Sydney,NSW,-33.848938,151.131197,2,unit,NSW3,NSW282,...,2018-12-12,2018-12,2018,7.815466,2.428571,1.327737e+06,760000.0,3600000.0,38,50454000.0
3,850000.0,Abbotsford,Sydney,NSW,-33.847565,151.129123,2,unit,NSW3,NSW282,...,2019-02-25,2019-02,2019,8.023985,2.428571,1.327737e+06,760000.0,3600000.0,38,50454000.0
4,1220000.0,Abbotsford,Sydney,NSW,-33.850851,151.129917,3,unit,NSW3,NSW282,...,2019-04-12,2019-04,2019,7.912232,2.428571,1.327737e+06,760000.0,3600000.0,38,50454000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27007,1316000.0,Watsons Bay,Sydney,NSW,-33.844504,151.283744,2,unit,NSW4222,NSW180,...,2019-11-14,2019-11,2019,6.466743,2.000000,1.316000e+06,1316000.0,1316000.0,1,1316000.0
27008,NaN,Carss Park,Sydney,NSW,-33.982144,151.118237,3,unit,NSW851,NSW325,...,2019-11-08,2019-11,2019,16.567139,3.000000,NaN,NaN,NaN,0,0.0
27009,NaN,Clemton Park,Sydney,NSW,-33.923007,151.101954,1,unit,NSW940,NSW332,...,2020-01-13,2020-01,2020,12.792880,1.000000,NaN,NaN,NaN,0,0.0
27010,290000.0,Yennora,Sydney,NSW,-33.865854,150.970347,1,unit,NSW4538,NSW315,...,2019-11-29,2019-11,2019,22.640103,1.500000,3.875000e+05,290000.0,485000.0,2,775000.0


In [155]:
features = df[['bedrooms', 'distance_from_cbd_km', 'sub_avg_bedrooms', 'sub_avg_price', 'sub_min_price', 'sub_max_price', 'sub_house_count', 'sub_total_sales', 'lon', 'lat']]
target = df['price']

# Calculate the correlation coefficients between the features and target variable
corr_matrix = features.corrwith(target)

# Create a new DataFrame to store the correlation coefficients
corr_df = pd.DataFrame({'Feature': corr_matrix.index, 'Correlation Coefficient': corr_matrix.values})

# Sort the DataFrame by correlation coefficient in descending order
corr_df = corr_df.sort_values('Correlation Coefficient', ascending=False)
corr_df

,Feature,Correlation Coefficient
3,sub_avg_price,0.609593
5,sub_max_price,0.475887
8,lon,0.449195
0,bedrooms,0.360261
4,sub_min_price,0.320580
7,sub_total_sales,0.281429
9,lat,0.019227
6,sub_house_count,0.009644
2,sub_avg_bedrooms,-0.074128
1,distance_from_cbd_km,-0.416576


In [156]:
#corr_df.to_csv('corr_df.csv', index=False)

In [157]:
#df.to_csv('df.csv', index=False)

In [158]:
#suburb_stats.to_csv('suburb_stats.csv', index=False)